In [1]:
import torch

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset, filter_agents_by_frames
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer

from l5kit.cle.closed_loop_evaluator import ClosedLoopEvaluator, EvaluationPlan
from l5kit.cle.metrics import (CollisionFrontMetric, CollisionRearMetric, CollisionSideMetric,
                               DisplacementErrorL2Metric, DistanceToRefTrajectoryMetric)
from l5kit.cle.validators import RangeValidator, ValidationCountingAggregator
from l5kit.simulation.dataset import SimulationConfig
from l5kit.simulation.unroll import ClosedLoopSimulator
import os
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from l5kit.visualization.visualizer.zarr_utils import simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.data import MapAPI

In [2]:
import gym
import l5kit.environment

# By setting the L5KIT_DATA_FOLDER variable, we can point the script
# to the folder where the data lies.
os.environ["L5KIT_DATA_FOLDER"] = os.environ["HOME"] + '/level5_data'
env_config_path = "./config.yaml"

In [5]:
# call
env = gym.make('L5-CLE-v0', env_config_path=env_config_path)

/home/ubuntu/src/l5kit/l5kit/l5kit/data/zarr_dataset.py:226: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/home/ubuntu/src/l5kit/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")
